# Metrowest machine learning self-driving car
```
The project is a self-driving car.
This notebook trains the the CNN.
Refer to the .pdf for details
  
Much of this is taken from Kiras CIFAR-10 example - see reference.

References:
https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py
https://www.meetup.com/Natick-Artificial-Intelligence-Meetup/
https://github.com/experiencor.self-driving-toy-car/
http://vision.stanford.edu/teaching/cs231n/
https://www.tensorflow.org/
```

In [85]:
# force libraries to re-load every time - helpful during development
%load_ext autoreload
%autoreload 2

In [86]:
import sys
sys.path.append('../lib')

# Data Loading

In [87]:
from metrowestcar_dataset import Dataset
dataset = Dataset.data()


In [88]:

num_classes = 4

# input image dimensions
img_rows, img_cols = 64, 64
input_shape = (1, img_rows, img_cols)

# the data, split between train and test sets
dataset = Dataset.data()
x_train_raw = dataset['images_train']
x_test_raw  = dataset['images_test']
y_train_raw = dataset['steering_train']
y_test_raw  = dataset['steering_test']

# dataset is uint8 but Kiras expects float - do this conversion
# also - convert class vectors to binary (1-hot) encoded array

x_train = x_train_raw.astype('float32')
x_test = x_test_raw.astype('float32')
x_train /= 255
x_test /= 255

y_train = keras.utils.to_categorical(y_train_raw, num_classes)
y_test = keras.utils.to_categorical(y_test_raw, num_classes)

print("x_train before:" + str(type(x_train_raw[0][0][0][0])))
print("x_train after: " + str(type(x_train[0][0][0][0])))
print("y_train before " + str(type(y_train_raw)) + " " + str(y_train_raw.shape) + " " +  str(type(y_train_raw[0])))
print("y_train after  " + str(type(y_train)) + " " + str(y_train.shape) + str(type(y_train[0][0])))

x_train before:<class 'numpy.uint8'>
x_train after: <class 'numpy.float32'>
y_train before <class 'numpy.ndarray'> (2582,) <class 'numpy.uint8'>
y_train after  <class 'numpy.ndarray'> (2582, 4)<class 'numpy.float32'>


## Construct the Model

In [89]:
'''Trains a CNN on the dataset.
'''

from __future__ import print_function

import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os


model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
activation_25 (Activation)   (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 62, 62, 32)        9248      
_________________________________________________________________
activation_26 (Activation)   (None, 62, 62, 32)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 31, 31, 64)        18496     
__________

# Train the model

In [90]:
batch_size = 32
epochs = 3
data_augmentation = False

save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'metrowest_keras_trained_model.h5'


# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

assert data_augmentation is False, "data augmentation is not coded"

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)


# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Train on 2582 samples, validate on 646 samples
Epoch 1/3
2582/2582 [==============================] - 193s 75ms/step - loss: 1.4695 - acc: 0.3803 - val_loss: 1.3131 - val_acc: 0.4443
Epoch 2/3
2582/2582 [==============================] - 247s 96ms/step - loss: 1.3428 - acc: 0.4094 - val_loss: 1.1251 - val_acc: 0.4009
Epoch 3/3
2582/2582 [==============================] - 197s 76ms/step - loss: 1.2420 - acc: 0.4136 - val_loss: 1.0371 - val_acc: 0.4303
Saved trained model at /home/chris/Documents/projects_scikit_tensorflow/test2/metrowest_scikit_tensorflow_cnn_car/model/saved_models/metrowest_keras_trained_model.h5 
646/646 [==============================] - 12s 19ms/step
Test loss: 1.0370822100447428
Test accuracy: 0.43034055727554177


In [46]:
# Plot loss as a function of iteration number

In [146]:
# apply an image to the trained CNN
from metrowestcar_display import Displayer
displayer = Displayer()

image = x_train_raw[5]
displayer.display_image(image)
image = image.astype('float32')
image /= 255

image = image[np.newaxis, :]
steering_out = model.predict(image)[0]
s_max = np.where(steering_out==steering_out.max())
s_index = s_max[0][0]
if s_index == 1:
    print("left")
elif s_index == 2:
    print("right")
elif s_index == 3:
    print("center")
else:
    assert False, "something wrong in prediction %r" % s_index


center
